## Logistics Regression (Using gradient ascent algorithm)
**Author**: Evi Ofekeze

This notebook implements the gradient ascent algorithm for logistics regression. Note that this is slighty different from the gradient descent algorithm. However, the follwoing code can be adapted for gradient descent by changing the sign. The mathetical formulation for the algorithm can be found in the appendix at the end of the document if you are into that kind of thing. However you can totally ignore the information in the appendix.


**Gradient Ascent Algorithm for Logistic Regression**

1. $gradw =$ all zeros (to compute the first gradient)
2. $gradb = 0$ (to compute the second gradient)
3. for *epoch* in $[1,2,...maxiter]$:
	1. for $x^{(i)},y^{(i}$ for $i \in [i, m]$ in training data:

		1. $gradw \leftarrow gradw$ + $x^{(i)} \cdot (y^{(i)} - h_\theta(x^{(i)})$
		2. $gradb \leftarrow gradb + (y^{(i)} - h_\theta(x^{(i)})$
		3. $w \leftarrow w + \eta gradw - \alpha w$
		4. $w \leftarrow w + \eta gradw$

In [92]:
class LogisticsRegression:
    
    """
    The Class contains a preedict and fit function for a logistics regression problem
    Input:
        Featrue matrix
        Outcome/Class vector
    Output:
        Model parameter
        Intercept
    """
    def __init__(self, learning_rate=0.01, max_iteration=1000, c = 0.1):
        self.learning_rate = learning_rate
        self.max_iteration = max_iteration
        self.c = c

    def __sigmoid(self, z):
        """
        The logit/sigmoid function for that return value between 0 and 1
        """
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        """
        A fit funtion for the algorithm above
        """
        
        self.weights = np.zeros((X.shape[1],1))
        self.bias = np.zeros((1,1))
        
        for time in range(self.max_iteration):
            z = np.dot(X, self.weights) + self.bias
            h = self.__sigmoid(z)
    
            gradweight = np.dot(X.T, (y - h))
            gradbias = np.sum((y - h))
           
            self.weights += self.learning_rate * (gradweight - (1/self.c) * self.weights)
            self.bias += self.learning_rate * gradbias
        
        self.coef_ = self.weights
        self.intercept_ = self.bias

    def predict_prob(self, X):
        """
        A predict function to return probabilities 
        """
        return self.__sigmoid(np.dot(X, self.weights) + self.bias)

    def predict(self, X): 
        """
        A predict funtion to place in their respective classes
        """
        return (self.predict_prob(X) > 0.5).astype(int)

## **Implentation**

The functinality of the algorithm will be tested using the iris dataset from sklearn library. We shall perform a binary classification by for setosa and others. The result will be compared to scikit learns logistics regression.

In [43]:
#Importing standard libraries
import numpy as np
import matplotlib.pyplot as plt

#Importing Machine Learning Libraries
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### **Data**

The data consist of the following:
- Outcome/class variable stored in the target.
- The class names are stored in the variable target_names.
- The predictor variables stored as in the variable data

Setosa is represented by as 0's, verisicolor and virginica is coded as 1' and 2's respectively. we will convert none zeros to ones for this binary classification. 

In [59]:
# Load the dataset
iris = load_iris()

# Feature variables
X = iris.data

# Target variables
y = (iris.target != 0) * 1
y = y.reshape(-1,1)

In [60]:
# It is customary to set aside some data for testing, this is no exception!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

In [61]:
# We standardized the data, to prevent domineering effects of large variable. 
# The following code ensures a mean of 0 and a standard deviation of 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [86]:
model = LogisticsRegression(learning_rate=0.01, max_iteration=100000, c = 1)
model.fit(X_train, y_train)

In [87]:
model.coef_

array([[ 1.04274213],
       [-1.12900293],
       [ 1.69946447],
       [ 1.54545672]])

In [77]:
model.intercept_

array([[2.37787162]])

## **Implentation: Using SkLearn**

In [78]:
from sklearn.linear_model import LogisticRegression

In [88]:
sk_model = LogisticRegression(max_iter= 100000)
sk_model = sk_model.fit(X_train,y_train.ravel())

In [91]:
sk_model.coef_

array([[ 1.04274427, -1.12899966,  1.69946559,  1.54545684]])

From the above results we see that both methods produced comparable results. It works!!!

No Machine Learning model is complete without testing on the test data.

In [124]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

predictions_test = model.predict(X_test)
prediction_sklearn = sk_model.predict(X_test)

test_df = pd.DataFrame({'Metric':['Scratch', 'SkLearn'],
                        'Accuracy':[accuracy_score(y_true = y_test, y_pred = predictions_test), accuracy_score(y_true = y_test, y_pred = prediction_sklearn)],
                       'Precision':[accuracy_score(y_true = y_test, y_pred = predictions_test), accuracy_score(y_true = y_test, y_pred = prediction_sklearn)],
                       'Recall':[accuracy_score(y_true = y_test, y_pred = predictions_test), accuracy_score(y_true = y_test, y_pred = prediction_sklearn)],
                       'F1 Score':[accuracy_score(y_true = y_test, y_pred = predictions_test), accuracy_score(y_true = y_test, y_pred = prediction_sklearn)],}).set_index('Metric')
test_df.T

Metric,Scratch,SkLearn
Accuracy,1.0,1.0
Precision,1.0,1.0
Recall,1.0,1.0
F1 Score,1.0,1.0


### **Appendix : Mathematical formulation**

Consider as Dataset represented by a feature matrix $\mathbf{x} = (x_{ij}, ...)$ , and an outcome vector $\mathbf{y} = \{0,1\}$

We assume a model of the form

$$P(y= 1|x) = \mathbf{w}^T \mathbf{x} + b$$
where $w = weights$ vector  and b is the bias term.


We optimise the maximum likehood function for the condtional probability of $y$ given $x$ and $\theta$

\begin{align}
\text{log} L(\theta) = \sum_{i=1}^{m} y_i \cdot \text{log} h_{\theta}(\mathbf x_i) + (1 - y_i) \cdot \text{log}(1 -  h_{\theta}((\mathbf x_i)) - \alpha \mathbf \theta^T \mathbf \theta
\end{align}


The gradient ascent approach gives us the following update rule

$$\theta^{(t)} \longrightarrow \theta^{(t-1)} + \eta \nabla log L(\theta) |_{\theta=\theta^{(t-q)}}$$

now computing $\nabla log L(\theta)$:

we have 

\begin{align}
\nabla_{}\text{log}(\theta) &= \nabla_{\theta} \sum^{m}_{i=1} y^{(i)} \cdot  \text{log} h_{\theta}(x^{(i)}) + (1-y^{(i)}) \cdot \text{log}(1-h_{\theta}x^{(i)})\\
&= \nabla_{\theta} \sum^{m}_{i=1} \frac{y^{(i)}}{h_{\theta}(x)}y^{(i)} \cdot \nabla_{\theta} h_{\theta}(x^{(i)}) + \frac{(1-y^{(i)})}{1-h_{\theta}(x^{(i)})} \cdot \nabla_{\theta}(1-h_{\theta}x^{(i)}) \tag{1}
\end{align}

Evaluting $\nabla_{\theta} h_{\theta}(x^{(i)})$ where $\theta$ 

we have

$$ h_{\theta}(\mathbf x_i) = \frac{1}{1 + e^{\mathbf w \mathbf x + b}}$$

Its derivative with respect to $\theta$ gives us

\begin{align}
&=\frac{-1}{(1 + e^{-(\textbf{wx} + b)})^2} \cdot e^{(\textbf{wx} + b)} \cdot -\textbf{x}\\
&= \textbf{x} \cdot \frac{1}{1 + e^{-(\textbf{wx} + b)}} \cdot \frac{{e^{-(\textbf{wx} + b)}}}{{1 + e^{-(\textbf{wx} + b)}}}\\
&= \textbf{x} \cdot \frac{1}{1 + e^{-(\textbf{wx} + b)}} \cdot \bigg(1- \frac{1}{{1 + e^{-(\textbf{wx} + b)}}} \bigg)\\
&= \mathbf x_i \cdot h_{\theta}(\mathbf x_i) \cdot ( 1 - h_{\theta}(\mathbf x_i))
\end{align}


Since 
$$ h_{\theta}(\mathbf x_i) = \frac{1}{1 + e^{\mathbf w \mathbf x + b}}$$


inputing  the value of $\nabla_{\theta} h_{\theta}(x^{(i)})$ in equation 1 and for simplicity represent instances of $x^i$ anf $y^i$ as  $x$ and $y$ and simply ask the obejective to penalise the $\textbf{L2}$ norm we have 

\begin{align} 
\nabla_{\theta} log L(\theta) &=  \sum_{i=1}^{m}  \frac{y_i}{h_{\theta}(\mathbf x_i)} \mathbf x_i \cdot h_{\theta}(\mathbf x_i) ( 1 - h_{\theta}(\mathbf x_i))(\mathbf x_i)  + \frac{(1 - y_i)}{(1 -  h_{\theta}) (- \mathbf x_i} \cdot h_{\theta}(\mathbf x_i) \cdot ( 1 - h_{\theta}(\mathbf x_i)) - \alpha\mathbf \theta \\
&= \sum_{i=1}^{m} y  \cdot  \textbf x(1-h_{\theta}(x)) + (y-1) \textbf x \cdot \dot h_{\theta}(\textbf x)\\
&= \sum_{i=1}^{m} \textbf x \cdot y (1-h_{\theta}(x)) + (y-1) \dot h_{\theta}(\textbf x)\\
& = \sum_{i=1}^m \mathbf x \cdot(y - h_{\theta}(\mathbf x)) - \alpha\mathbf \theta
\end{align}


giving us an update rule of 

$$ \theta_{i+1} = \theta_i + \sum_{i=1}^m \mathbf x_i \cdot(y - h_{\theta}(\mathbf x_i)) - \alpha\mathbf \theta$$

Thus the gradient ascent for the weigths and the bias can be written as 

$$ w_{i+1} \longrightarrow w_i + \sum_{i=1}^m \mathbf x_i \cdot(y - h_{\theta}(\mathbf x_i)) - \alpha\mathbf \theta \tag{3}$$

$$ b_{i+1} \longrightarrow b_i + \sum_{i=1}^m \mathbf x_i \cdot(y - h_{\theta}(\mathbf x_i)) \tag{4}$$